In [16]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
from pyproj import Transformer
from tqdm import tqdm
import numpy as np

In [17]:
caracteristiques_raw = pd.read_csv("data/accidents/caracteristiques.csv", encoding="latin1", low_memory=False)
usagers_raw = pd.read_csv("data/accidents/usagers.csv", encoding="latin1", low_memory=False)

## Característics cleaning

In [18]:
caracteristiques = caracteristiques_raw.copy()

In [19]:
def add_zeroes(x):
	if len(str(x)) >= 4:
		return str(x)
	else:
		return add_zeroes("0" + str(x))
caracteristiques['hrmn'] = caracteristiques['hrmn'].apply(lambda x: add_zeroes(x))

In [20]:
caracteristiques['hrmn'] = caracteristiques['hrmn'].astype(str)
caracteristiques['hrmn'] = caracteristiques['hrmn'].apply(lambda x: x[:2] + ":" + x[2:] if len(x) == 4 else x)

In [21]:
caracteristiques["dep"] = caracteristiques["dep"].astype(str)
caracteristiques["dep"] = caracteristiques["dep"].apply(lambda x: x[:2])

In [22]:
caracteristiques['lat'] = pd.to_numeric(caracteristiques['lat'].str.replace(',', '.').replace('', pd.NA).dropna(), errors='coerce')
caracteristiques['long'] = pd.to_numeric(caracteristiques['long'].str.replace(',', '.').replace('', pd.NA).dropna(), errors='coerce')

In [23]:

# Create a transformer object to convert Lambert-93 (EPSG:2154) to WGS84 (EPSG:4326)
transformer = Transformer.from_crs("EPSG:2154", "EPSG:4326", always_xy=True)


# Function to check if the value is in a projected format (i.e., large numbers typical for projected coordinates in meters)
def is_projected_coordinate(lat, long):
    # Using a more typical threshold for projected coordinates (e.g., any values outside typical lat/long ranges)
    return abs(lat) > 10000 or abs(long) > 10000  # Adjust based on typical Lambert-93 ranges

# Function to convert Lambert-93 to WGS84
def convert_projected_to_latlong(lat, long):
    if pd.notna(lat) and pd.notna(long):
        long_wgs84, lat_wgs84 = transformer.transform(long, lat)
        return lat_wgs84, long_wgs84
    return None, None

# Iterate over the rows and convert the coordinates if they are in a projected format
for index, row in tqdm(caracteristiques.iterrows(), total=caracteristiques.shape[0]):
    if is_projected_coordinate(row['lat'], row['long']):
        new_lat, new_long = convert_projected_to_latlong(row['lat'], row['long'])
        if new_lat and new_long:
            caracteristiques.at[index, 'lat'] = new_lat
            caracteristiques.at[index, 'long'] = new_long

# Optionally, remove any rows that still have invalid lat/long values
caracteristiques.dropna(subset=['lat', 'long'], inplace=True)


100%|██████████| 1121571/1121571 [01:17<00:00, 14389.50it/s]


In [24]:
# Remove the 'Unnamed: 0' column
caracteristiques = caracteristiques.drop(columns=[col for col in ['Unnamed: 0', "adr", "gps", 'com'] if col in caracteristiques.columns])
caracteristiques['num_acc'] = caracteristiques['num_acc'].astype(int).astype(str)


In [25]:
caracteristiques

,num_acc,mois,jour,hrmn,lum,agg,int,atm,col,lat,long,dep,annee
0,200500000001,1,12,19:00,3,2,1,1.0,3.0,32.573415,-1.207836,59,2005
1,200500000002,1,21,16:00,1,2,1,1.0,1.0,32.585945,-1.356116,59,2005
2,200500000003,1,21,18:45,3,1,1,2.0,1.0,32.593739,-1.358702,59,2005
3,200500000004,1,4,16:15,1,1,1,1.0,5.0,32.960637,-1.792306,59,2005
4,200500000005,1,10,19:45,3,1,1,3.0,6.0,32.943998,-1.721118,59,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121566,202100056514,1,1,06:10,3,1,1,5.0,6.0,44.526619,-0.195544,33,2021
1121567,202100056515,1,1,10:20,1,1,1,2.0,6.0,45.380291,5.704914,38,2021
1121568,202100056516,1,1,18:00,3,1,1,2.0,1.0,44.911210,5.019636,26,2021
1121569,202100056517,1,1,10:55,1,1,2,1.0,6.0,44.954275,-0.517921,33,2021


## Usagers

In [26]:
usagers = usagers_raw.copy()

In [27]:
usagers = usagers.drop(columns=[col for col in ['num_veh', "id_vehicule", "secu1", "secu2", "secu3", "secu", "locp", "actp", "etatp", "Unnamed: 0"] if col in usagers.columns])
usagers['num_acc'] = usagers['num_acc'].astype(int).astype(str)

In [28]:
usagers

,num_acc,place,catu,grav,sexe,trajet,an_nais,annee
0,200500000001,1.0,1,4,1,1.0,1976.0,2005
1,200500000001,1.0,1,3,2,3.0,1968.0,2005
2,200500000001,2.0,2,1,1,0.0,1964.0,2005
3,200500000001,4.0,2,1,1,0.0,2004.0,2005
4,200500000001,5.0,2,1,1,0.0,1998.0,2005
...,...,...,...,...,...,...,...,...
2509615,202100056516,1.0,1,4,1,0.0,1975.0,2021
2509616,202100056516,1.0,1,4,1,5.0,2002.0,2021
2509617,202100056517,1.0,1,3,1,0.0,1999.0,2021
2509618,202100056518,1.0,1,3,1,3.0,1968.0,2021


In [29]:
# Save the cleaned data
caracteristiques.to_csv("data/accidents/caracteristiques_cleaned.csv", index=False)
usagers.to_csv("data/accidents/usagers_cleaned.csv", index=False)

In [30]:
# save to parquet
caracteristiques.to_parquet("data/accidents/caracteristiques_cleaned.parquet")
usagers.to_parquet("data/accidents/usagers_cleaned.parquet")

In [31]:
caracteristiques['agg'].value_counts()

agg
2    350090
1    281200
Name: count, dtype: int64